In [ ]:
# create a default profile
ipython profile create
# turn off the capture flag
echo "c.IPKernelApp.capture_fd_output = False" >> \
     "~/.ipython/profile_default/ipython_kernel_config.py"

In [1]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [2]:
sc

<SparkContext master=local[2] appName=PySparkShell>

In [3]:
spark

In [6]:
import random
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.tuning import CrossValidator, CrossValidatorModel

In [8]:
globals()['models_loaded'] = False
globals()['my_model'] = CrossValidatorModel.load('/Users/christianbutcher/Desktop/spark/model')

In [14]:
def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model'] = CrossValidatorModel.load('/Users/christianbutcher/Desktop/spark/model') 
        # Replace '***' with:    [...].load('my_logistic_regression')
        globals()['models_loaded'] = True

    
    df_result = globals()['my_model'].transform(df)
    df_result.select('app_id','label','review_id','review_text','prediction').show()

In [15]:
ssc = StreamingContext(sc, 10)

In [16]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [17]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

23/05/04 15:26:05 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/04 15:26:05 WARN BlockManager: Block input-0-1683206764800 replicated to only 0 peer(s) instead of 1 peers
23/05/04 15:26:09 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/04 15:26:09 WARN BlockManager: Block input-0-1683206768800 replicated to only 0 peer(s) instead of 1 peers


========= 2023-05-04 15:26:10 =========


+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|1774060|    0|137772059|I love ball and p...|
|2359850|    1|137773694|I died 50 times i...|
+-------+-----+---------+--------------------+

23/05/04 15:26:12 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/04 15:26:12 WARN BlockManager: Block input-0-1683206771800 replicated to only 0 peer(s) instead of 1 peers
23/05/04 15:26:16 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/04 15:26:16 WARN BlockManager: Block input-0-1683206775800 replicated to only 0 peer(s) instead of 1 peers


23/05/04 15:26:19 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/04 15:26:19 WARN BlockManager: Block input-0-1683206779000 replicated to only 0 peer(s) instead of 1 peers


23/05/04 15:26:21 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/04 15:26:21 WARN BlockManager: Block input-0-1683206780800 replicated to only 0 peer(s) instead of 1 peers
23/05/04 15:26:24 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/04 15:26:24 WARN BlockManager: Block input-0-1683206784000 replicated to only 0 peer(s) instead of 1 peers


...

23/05/04 15:26:31 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/04 15:26:31 WARN BlockManager: Block input-0-1683206790800 replicated to only 0 peer(s) instead of 1 peers
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|2359850|    1|137772961|Nice, clean, fun ...|
|2266010|    1|137771099|          Good game.|
|1277920|    1|137773738|idk what everyone...|
+-------+-----+---------+--------------------+



+-------+-----+---------+--------------------+----------+
| app_id|label|review_id|         review_text|prediction|
+-------+-----+---------+--------------------+----------+
|2359850|    1|137772961|Nice, clean, fun ...|       1.0|
|2266010|    1|137771099|          Good game.|       1.0|
|1277920|    1|137773738|idk what everyone...|       1.0|
+-------+-----+---------+--------------------+----------+

========= 2023-05-04 15:26:30 =========
23/05/04 15:26:34 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/04 15:26:34 WARN BlockManager: Block input-0-1683206793800 replicated to only 0 peer(s) instead of 1 peers
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|1277920|    0|137772592|full of glitches ...|
|1277920|    0|137772459|What a waste of t...|
|1277920|    0|137771483|Don't even instal...|
+-------+-----+---------+--------------------+



+-------+-----+---------+--------------------+----------+
| app_id|label|review_id|         review_text|prediction|
+-------+-----+---------+--------------------+----------+
|1277920|    0|137772592|full of glitches ...|       1.0|
|1277920|    0|137772459|What a waste of t...|       0.0|
|1277920|    0|137771483|Don't even instal...|       0.0|
+-------+-----+---------+--------------------+----------+



========= 2023-05-04 15:26:40 =========
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|1294810|    0|137774101|So this was a tou...|
|1294810|    0|137773593|Dont buy this pls...|
+-------+-----+---------+--------------------+



+-------+-----+---------+--------------------+----------+
| app_id|label|review_id|         review_text|prediction|
+-------+-----+---------+--------------------+----------+
|1294810|    0|137774101|So this was a tou...|       0.0|
|1294810|    0|137773593|Dont buy this pls...|       0.0|
+-------+-----+---------+--------------------+----------+



In [18]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
23/05/04 15:26:44 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
23/05/04 15:26:44 WARN SocketReceiver: Error receiving data
java.net.SocketException: Socket closed
	at java.base/java.net.SocketInputStream.socketRead0(Native Method)
	at java.base/java.net.SocketInputStream.socketRead(SocketInputStream.java:115)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:168)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:140)
	at java.base/sun.nio.cs.StreamDecoder.readBytes(StreamDecoder.java:284)
	at java.base/sun.nio.cs.StreamDecoder.implRead(StreamDecoder.java:326)
	at java.base/sun.nio.cs.StreamDecoder.read(StreamDecoder.java:178)
	at java.base/java.io.InputStreamReader.read(InputStreamReader.java:181)
	at java.base/java.io.BufferedReader.fill(BufferedReader.java:161)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:326)
	at java.base/java.io.BufferedRea

Exception in thread "receiver-supervisor-future-0" java.lang.InterruptedException: sleep interrupted
	at java.base/java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.ja